In [69]:
import numpy as np
import math
from sympy import *
import scipy.optimize as optimize

In [70]:
tensForceMax = 12
compForceMax = 8
costPerM = 15

In [71]:
def toVector(v):
    return np.hstack([v.flatten()])

In [72]:
def toMatrix(vec):
    return np.reshape(vec, (5,2))

In [73]:
def solveTruss(elementsInsert):

    elements2np = toMatrix(elementsInsert)
    elements2 = elements2np.tolist() #list for non-fixed points below road

    #inputs

    #fixed road points, change accordingly, [x,y] format
    elements1 = [[0,0],[3,0],[6,0],[9,0],[12,0],[15,0]]

    #ops on inputs
    elements = elements1 + elements2 #joint numbers are based on their position in this list e.g. joint 0 is elements[0]

    #connections between joints [joint1, joint2].
    joints=[[0,1],[1,2],[2,3],[3,4],[4,5],[5,10],[10,8],[8,6],[6,7],[7,9],[9,0],[1,9],[1,7],[7,2],[2,6],[6,3],[3,8],[8,4],[4,10]]

    #loads from left to right. Adjust accordingly
    loads = [-7.5/2,-15/2,-15/2,-15/2,-15/2,-7.5/2]

    #supports, ask thomas how this works lol
    supports = [["s",0,0],["s",0,1],["s",5,1]]

    

    combined = joints + supports
    size = (len(elements)*2, len(elements)*2)
    matrix = np.zeros(size)
    lenTooSmall = False
    distances = []

    for i in range(len(combined)):
        if(len(combined[i]) == 2):
            diffX = elements[combined[i][1]][0]-elements[combined[i][0]][0]
            diffY = elements[combined[i][1]][1]-elements[combined[i][0]][1]

            distance = np.sqrt(diffX*diffX + diffY*diffY)
            if(distance < 1):
                lenTooSmall = True
                break
            else:
                distances.append(distance)
            
            angle = np.arctan2(diffY, diffX)
            matrix[2*combined[i][0]][i] = np.cos(angle)
            matrix[2*combined[i][0]+1][i] = np.sin(angle)
            matrix[2*combined[i][1]][i] = np.cos(angle + np.pi)
            matrix[2*combined[i][1]+1][i] = np.sin(angle + np.pi)
        else:
            if combined[i][1] == 0:
                matrix[2*combined[i][1]][i] = 1
            if combined[i][2] == 1:
                matrix[2*combined[i][1]+1][i] = 1
    
    if(lenTooSmall):
        return 100000000

    forces = np.zeros(len(elements)*2)
    for i in range(len(loads)):
        forces[2*i+1] = loads[i]
    try:
        answer = np.linalg.solve(matrix,forces)
    except:
        return 100000000
    # print(solvedEquation)
    # print(distances, len(distances))
    totalCost = 0

    for i in range(len(answer)-3):
        memberForce = answer[i]
        memberValue = "(T)" if memberForce < 0 else "(C)"
        memberForce = -1*memberForce
        if i < len(elements)*2:
            numJoints = (math.ceil(memberForce/tensForceMax) if memberForce > 0 else math.ceil(-memberForce/compForceMax))
            if numJoints > 4:
                return 100000000
            memberCost = distances[i] * costPerM * numJoints
            totalCost += memberCost
            # print(distances[i], memberCost, memberForce)
        # print("Member " , i , ":", memberForce, memberValue, numJoints)

    totalCost += len(elements)*5
    return totalCost

In [74]:
x1 = 4.5
y1 = -4
x2 = 1.5
y2 = -2
y3 = -5
maxWidth = 15

#[x,y] format, example input to solveTruss function
non_fixed_pts = np.array([[7.5,-5],[4.5,-4],[10.5,-4],[1.5,-2],[13.5,-2]])
non_fixed_pts = np.array(toVector(non_fixed_pts), dtype=float) #<----- this is what will be inputted i.e. a flattened matrix

In [75]:

bounds = [(0, 15), (-50, 0), (0, 15), (-50, 0), (0, 15), (-50, 0), (0, 15), (-50, 0), (0, 15), (-50, 0),]
#add more bounds for each point you add. (0,15) for x, (-50, 0 for y). For ex, adding another pair would result in adding (0, 15), (-50, 0), to the end of the list.

result = optimize.differential_evolution(solveTruss, bounds)

In [76]:
print(result.x)
#results are read x y x y x y etc. 
#the results are the non-fixed points below the road. 

[ 7.62902847 -4.22805163  2.47344956 -2.64173768 12.41646119 -2.76486058
  0.44982925 -0.93635582 14.40520817 -1.1440536 ]


In [77]:
solveTruss(result.x)

1382.9146568235892

In [78]:
# lowestCost = 1000000
# points = []

# for i in range(100):
#     print(i, end=" ")
#     result = optimize.differential_evolution(solveTruss, bounds)
#     currCost = solveTruss(result.x)
#     print(currCost)
#     if(currCost < lowestCost):
#         lowestCost = currCost
#         points = result.x
#         print(lowestCost, points)

0 1379.8059449633015
1379.8059449633015 [ 7.4160194  -4.3140465   2.36637898 -2.5224016  12.57543623 -2.54663712
  0.49635319 -0.96126099 14.50113469 -0.94285283]
1 1389.3982122762086
2 1398.8181838727403
3 1387.8536378510516
4 1406.6467754020869
5 1397.0761685251787
6 1393.4730933083954
7 1392.6185455563605
8 1396.9625239371353
9 1400.8599563741368
10 1387.1819075298472
11 1405.1965193216665
12 1390.5516499108996
13 1390.5166719829276
14 1391.9476963879956
15 1395.6322860091839
16 1387.149556959675
17 1380.8822392087302
18 1408.2870185243385
19 1389.6943777178894
20 1399.8972282477841
21 1391.0567876321957
22 1397.792274608095
23 1392.3561423575768
24 1384.0528720023083
25 1392.1783488201309
26 1388.3871240444228
27 1387.4791389359834
28 1383.1464882227203
29 1382.2724300878783
30 1383.7171022076964
31 1389.8779589538706
32 1387.6800365547508
33 1394.3791068380226
34 1412.2699851243565
35 1401.3105415784207
36 1395.0795003359433
37 1394.3781145472947
38 1395.0172063691166
39 1386.3452